---
comments: True
layout: post
title: Retrospective and Review of William's Game
description: Review and Key points from William's Game level
permalink: /csse/javascript/Retrospective_Reflection
categories: [PBL_Blogs]
author: Akhil Kulkarni
---

For William's game level, I was especially impressed with his quest implementation in his game level, and how using conditions, he was able to spawn and render new quests/jobs incrementally as the user completed each task job by job. Very nice implementation overall! Below is a code snippet of hoow he does this and what I really liked about his changes:

In [ ]:
 static interactionChecks(objectID) {
        // This function gets called repeatedly per second
        for (let i = 0; i < this.quests.length; i++) {
            let quest = this.quests[i];
            if (quest.Activated === false) {
                continue;
            }

            if (quest.Type === "NPCtalks") {
                Quest.updateQuest(quest, objectID);
            }
            if(quest.Type === "Scavenger"){
                Quest.updateQuest(quest, objectID)
            }

        }
    }

    static message(text, goBacktoOGText){
        const TextBox = document.getElementById("questMessage")
        const originalText = TextBox.innerHTML
        TextBox.innerHTML = text
        if(goBacktoOGText){
            setTimeout(() => {
                TextBox.innerHTML = originalText
            }, 2000);
        }
    }


1️⃣ interactionChecks(objectID)
Purpose: This function is called repeatedly (likely every frame or at a fixed interval per second) to check for interactions between the player and active quests.

How it works:

Loops through all quests stored in this.quests.
If a quest is not activated (Activated === false), it skips that quest.
If the quest type is "NPCtalks" or "Scavenger", it calls Quest.updateQuest(quest, objectID), updating the quest progress based on the interaction.
Assumptions:

this.quests is an array containing all quest objects.
Quest.updateQuest(quest, objectID) is another function (defined elsewhere) that updates the quest when interacting with the given object.
2️⃣ message(text, goBacktoOGText)
Purpose: Displays a message in a div and optionally restores the original text after 2 seconds.

How it works:

Finds an HTML element with id="questMessage" (this is where the quest messages are shown).
Saves its original text.
Replaces the text with the text argument.
If goBacktoOGText is true, it sets a 2-second timeout to restore the original message.
Use case:

Example: If a player talks to an NPC, this method can show "You found a clue!" for 2 seconds and then revert to "Find more clues!".
Summary
✅ interactionChecks(objectID):

Continuously checks active quests and updates them based on the player's interaction with an object.
✅ message(text, goBacktoOGText):

Updates the quest message in the UI and optionally resets it after 2 seconds.

He also added a new file called QuestNPC.js to render the quest popups and handle the main logic for his quest game:

In [ ]:
import GameEnv from "./GameEnv.js";
import Character from "./Character.js";
import Player from './Player.js';
import QuestSystem from "./QuestSystem.js";
import Quests from "./Quests.js";
import GameControl from "./GameControl.js";
import ScavengerObject from "./ScavengerObject.js";

class QuestNpc extends Character {
    constructor(data = null) {
        super(data);
        this.bindEventListeners();
    }

    update() {
        this.draw();
    }

    /**
     * Bind key event listeners for proximity interaction.
     */
    bindEventListeners() {
        addEventListener('keydown', this.handleKeyDown.bind(this));
        addEventListener('keyup', this.handleKeyUp.bind(this));
    }

    /**
     * Handle keydown events for interaction.
     * @param {Object} event - The keydown event.
     */
    handleKeyDown({ key }) {
        switch (key) {
            case 'e': // Player 1 interaction
            case 'u': // Player 2 interaction
                // Add logic to give the player random quests
                if (this.isPlayerNear() && this.areAnyQuestActive() === false) {
                    let randQuest = this.findRandomQuest(this.listOfDeactivatedQuests());
                    this.assignQuest(randQuest);
                } else {
                    if (this.isPlayerNear() === true) {
                        if (this.areAnyQuestActive()) {
                            QuestSystem.message("Finish your current quest first!", true);
                            console.log("Finish your current quest");
                        }
                    }
                }
                break;
        }
    }

    /**
     * Handle keyup events to stop player actions.
     * @param {Object} event - The keyup event.
     */
    handleKeyUp({ key }) {
        if (key === 'e' || key === 'u') {
            // Clear any active timeouts when the interaction key is released
            if (this.alertTimeout) {
                clearTimeout(this.alertTimeout);
                this.alertTimeout = null;
            }
        }
    }

    /**
     * Check if the player is near the NPC.
     * @returns {boolean} - True if the player is near the NPC, false otherwise.
     */
    isPlayerNear() {
        let objects = GameEnv.gameObjects;
        for (let i = 0; i < objects.length; i++) {
            let object = objects[i];
            if (object instanceof Player) {
                this.isCollision(object);
                return this.collisionData.hit;
            }
        }
        return false;
    }

    findRandomQuest(deactivatedQuests) {
        if (deactivatedQuests.length === 0) {
            return false;
        }
        function findRandIndex() {
            let randIndex = Math.floor(Math.random() * deactivatedQuests.length);
            let quest = deactivatedQuests[randIndex];

            if (quest.Activated === false) {
                return quest;
            }
        }
        return findRandIndex();
    }

    assignQuest(quest) {
        if (quest === false || quest === undefined) {
            QuestSystem.message(`I have no more quests for you`);
            return;
        }
        if (quest.Type === "Scavenger") {this.addScavengerQuestObject(quest.TypeOValues.itemsToFind);}
        quest.Activated = true;
        console.log(`Quest assigned: ${quest.Name}`);
        console.log(QuestSystem.quests);
        this.sendMessage(quest);
    }

    listOfDeactivatedQuests() {
        let deactivatedQuests = [];
        for (let i = 0; i < QuestSystem.quests.length; i++) {
            if (QuestSystem.quests[i].Activated === false && QuestSystem.quests[i].Completed === false) {
                deactivatedQuests.push(QuestSystem.quests[i]);
            }
        }
        console.log("got list of deactivated quests");
        return deactivatedQuests;
    }

    areAnyQuestActive() {
        for (let i = 0; i < QuestSystem.quests.length; i++) {
            if (QuestSystem.quests[i].Activated === true) {
                return true;
            }
        }
        return false;
    }

    sendMessage(quest) {
        const type = quest.Type;
        let text = "";

        if (type === "Scavenger") {
            text = this.scavengerMessage(quest);
            QuestSystem.message(text);
        }
        if (type === "NPCtalks") {
            text = this.npcQuestMessage(quest);
            QuestSystem.message(text);
        }
    }

    scavengerMessage(quest) {
        let text = `Can bring me ${quest.TypeOValues.itemsToFind} I need the ASAP!!!`;
        return text;
    }

    npcQuestMessage(quest) {
        console.log(quest);
        let text = `Can you talk to ${quest.TypeOValues.NPCsToTalkTo} for me?`;
        return text;
    }

    addScavengerQuestObject(id) {
        ScavengerObject.getRandomObject(id);
    }
}

export default QuestNpc;

What Each Method Does
constructor(data = null)

Calls the Character class constructor.
Initializes the NPC and binds event listeners for interaction.
update()

Calls this.draw() (likely renders the NPC on screen).
bindEventListeners()

Listens for key presses (keydown, keyup) to allow interaction.
handleKeyDown(event)

If the player presses 'e' or 'u' and is near the NPC:
If no quests are active → Assigns a random quest.
Otherwise, displays a message: "Finish your current quest first!"
handleKeyUp(event)

If 'e' or 'u' is released, clears any alert timeouts.
isPlayerNear()

Checks if the player is near the NPC using collision detection.
findRandomQuest(deactivatedQuests)

Picks a random inactive quest from the list.
assignQuest(quest)

Activates and assigns the quest to the player.
If it's a Scavenger quest, adds objects to collect.
Sends a message about the quest.
listOfDeactivatedQuests()

Returns a list of quests that are inactive and incomplete.
areAnyQuestActive()

Checks if any quests are currently active.
sendMessage(quest)

Displays a message based on the type of quest (Scavenger or NPCtalks).
scavengerMessage(quest)

Creates a quest message for scavenger quests.
npcQuestMessage(quest)

Creates a message for NPC interaction quests.
addScavengerQuestObject(id)

Retrieves a random scavenger object needed for the quest.


This code makes an NPC assign quests and respond to player interactions dynamically in the game. 

One improvement I would love to see in his game would be an urge to complete the quest. He could have implemented a timer for the user, or made it so the game level would not just end with the esc key if all the quests were not completed, which I believe would be conditional logic to be added in GameControl.js. 